In [1]:
from imutils.video import VideoStream 
import argparse, datetime, imutils 
import time, cv2 , numpy as np

video = "traffic02.mp4"
min_area = 500;

## Implementation with: 
1. The model frame is set as the first frame of the model. Not a good detection algorithm as system is subject to all movement in the video including those of minor shakes inthe tree. 


In [ ]:
# Get First Frame and use as background model 
vs = cv2.VideoCapture(video) #args["video"] 
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3))
firstFrame = None; DESIRED_WIDTH = 750; 

while True:
    frame = vs.read() 
    frame = frame[1] 
    
    if (firstFrame is None) and (frame is None):  #unable to read video
        print("ERROR: Unable to read video")
        break; 

    if frame is None: #End of video
        break;   
    
    frame = imutils.resize(frame, width=DESIRED_WIDTH) 
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) #convert to grayscale 
    gray = cv2.GaussianBlur(gray, (21, 21), 0) #Blur to minimize noise # compute the absolute difference between the current frame and # first frame 
    
    #Assign first frame
    if firstFrame is None:
        firstFrame = gray; 
        continue; 
        
    frameDelta = cv2.absdiff(firstFrame, gray) 
    thresh = cv2.threshold(frameDelta, 25, 255, cv2.THRESH_BINARY)[1] 
    # dilate the thresholded image to fill in holes, then find contours # on thresholded image
    
    thresh = cv2.dilate(thresh, None, iterations=2) 
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) 
    cnts = imutils.grab_contours(cnts) # loop over the contours 
    for index, c in enumerate(cnts): # if the contour is too small, ignore it 
        if cv2.contourArea(c) < 10: #args["min_area"]: 
            continue 
        # compute the bounding box for the contour, draw it on the frame, 
        # and update the text 
        (x, y, w, h) = cv2.boundingRect(c) 
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2) 
        cv2.putText(frame, "{}".format(index), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), index) 

    # perform connected component labelling, using propagated
    cv2.imshow("Traffic Video", frame)  #show video
    if (cv2.waitKey(1) & 0xFF) == ord("q"):  #exit if user presses 'q'
        break 
    #firstFrame = gray 
    
# exit video player and close all windows
vs.release() 
cv2.destroyAllWindows()

## Implement with moving average
1. Base model is a running average of last n secs
2. Observe how model eliminates moving trees

In [3]:
from imutils.video import VideoStream 
import argparse, datetime, imutils 
import time, cv2 , numpy as np

# Get OpenCV version and Properties
def get_fps(video):
    import cv2
    vs = cv2.VideoCapture(video);
    (major_ver, minor_ver, subminor_ver) = (cv2.__version__).split('.') # Find OpenCV version

    if int(major_ver)  < 3 :
        fps = vs.get(cv2.cv.CV_CAP_PROP_FPS)
        print ("Frames per second using video.get(cv2.cv.CV_CAP_PROP_FPS): %i"%(fps))
    else :
        fps = vs.get(cv2.CAP_PROP_FPS)
        print ("Frames per second using video.get(cv2.CAP_PROP_FPS) : %i"%(fps))
    vs.release(); 
    return fps

video = "traffic02.mp4"
fps = get_fps(video)

AVG_SECS = 5; fps = int(fps);
MODEL_FRAMES = fps*AVG_SECS;
model = [];
frame = None; index = 0; background_done = False

cur_frame, prev_frame = None, None

# Operate Actual Model
vs = cv2.VideoCapture(video) #args["video"] 
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3))
firstFrame = None; DESIRED_WIDTH = 750; min_area = 300


while True:
    frame = vs.read()[1]
    frame = imutils.resize(frame, width=DESIRED_WIDTH)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (21, 21), 0)
    
    prev_frame = cur_frame
    cur_frame = gray
            #Assign first frame
    if firstFrame is None:
        firstFrame = gray; 
        continue; 
        
    if background_done is False: 
        model.append(cur_frame);
        index += 1
        
        if index == MODEL_FRAMES: 
            print("Captured Background Model");
            background_done = True
            index = 0;
        continue
        
    model[index] = prev_frame;  index += 1
    if index == MODEL_FRAMES:
        index = 0;
     
    #Assign first frame
    avg_frame = cv2.convertScaleAbs(np.average(model, axis=0))
           
    frameDelta = cv2.absdiff(avg_frame, cur_frame) #(cv2.convertScaleAbs(firstFrame), cur_frame) 
    thresh = cv2.threshold(frameDelta, 25, 255, cv2.THRESH_BINARY)[1] 
    # dilate the thresholded image to fill in holes, then find contours # on thresholded image
    
    thresh = cv2.dilate(thresh, None, iterations=2) 
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) 
    cnts = imutils.grab_contours(cnts) # loop over the contours 
    for ind, c in enumerate(cnts): # if the contour is too small, ignore it 
        if ((cv2.contourArea(c) < 100)): # or (cv2.contourArea(c) > 10000)) : #args["min_area"]: 
            continue 
        # compute the bounding box for the contour, draw it on the frame, 
        # and update the text 
        (x, y, w, h) = cv2.boundingRect(c) 
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2) 
        cv2.putText(frame, "{}".format(ind), (x, y), cv2.FONT_HERSHEY_SIMPLEX, .5, (0, 0, 0), 2) 

    # perform connected component labelling, using propagated
    cv2.imshow("Traffic Video", frame)  #show video
    #cv2.imshow("Average Frame", avg_frame)
   # cv2.imshow("Traffic Difference", frameDelta)  #show video
    #cv2.imshow("Traffic Thresh", thresh)  #show video
    if (cv2.waitKey(1) & 0xFF) == ord("q"):  #exit if user presses 'q'
        break
    
# exit video player and close all windows
vs.release() 
cv2.destroyAllWindows()

Frames per second using video.get(cv2.CAP_PROP_FPS) : 29
Captured Background Model


## Differencing Using Accumulated weight

In [ ]:
vs = cv2.VideoCapture(video) #args["video"] 
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3))
firstFrame = None; DESIRED_WIDTH = 750; 


# Initialize average: 
fps = int(get_fps(video)); AVG_SECS = 5;
MODEL_FRAMES = fps*AVG_SECS;
model = np.zeros(MODEL_FRAMES);
frame = None;

avg = None
prev_frame, cur_frame = None, None
  
while True:
    frame = vs.read() 
    frame = frame[1]
    
    frame = imutils.resize(frame, width=DESIRED_WIDTH)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (21, 21), 0)
    
    prev_frame = cur_frame
    cur_frame = gray
    
    if (avg is None): 
        if prev_frame is None: 
            continue
        
        print("Capturing Background Model");
        avg = prev_frame.copy().astype("float")
        continue
        
    # compute the absolute difference between the current frame and # first frame 
    frameDelta = cv2.absdiff(gray, cv2.convertScaleAbs(avg))
    DELTA_THRESH = 25;
    thresh = cv2.threshold(frameDelta, DELTA_THRESH, 255, cv2.THRESH_BINARY)[1]
    
    # dilate the thresholded image to fill in holes, then find contours # on thresholded image
    thresh = cv2.dilate(thresh, None, iterations=2) 
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) 
    cnts = imutils.grab_contours(cnts) # loop over the contours 
    
    prev_cnts = None;
    for i, c in enumerate(cnts): # if the contour is too small, ignore it 
        if cv2.contourArea(c) < 100: #args["min_area"]: 
            continue 

        #print("Im before contor")
        if prev_cnts == None: #initialize the car id dictionary
            prev_cnts = {index:c for index, c in enumerate(cnts)}
            continue
            
        #print("Im in here now! Contour")
        similarity = {key : cv2.matchShapes(c, prev_cnts[key], 2, 0) for key in prev_cnts.keys()}
        temp = min(similarity.values()) 
        min_key = [key for key in similarity if similarity[key] == temp]
        
        (x, y, w, h) = cv2.boundingRect(c) 
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2) 
        cv2.putText(frame, "{}".format(min_key), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), thickness=2)
    
    
    
    cv2.imshow("Traffic Video", frame)  #show video
    if (cv2.waitKey(1) & 0xFF) == ord("q"):  #exit if user presses 'q'
        break 

# exit video player and close all windows
vs.release() 
cv2.destroyAllWindows()


In [ ]:
print(a[0])
a[0] = [10,19,20]
print(a)
print(np.average(a, axis=0))

In [ ]:
## Back up first frame as model

In [ ]:
vs = cv2.VideoCapture(video) #args["video"] 
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3))
firstFrame = None; DESIRED_WIDTH = 750; 


# Initialize average: 
fps = 30; AVG_SECS = 5;
MODEL_FRAMES = fps*AVG_SECS;
model = np.zeros(MODEL_FRAMES);
frame = None;

avg = None
motioncounter = 0;
prev_frame, cur_frame, next_frame = None, None, None
  
while index < MODEL_FRAMES:
    frame = vs.read() 
    frame = frame[1]
    
    frame = imutils.resize(frame, width=DESIRED_WIDTH)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (21, 21), 0)
    
    if background_done is False: 
        model[index] = gray;
        index += 1
        if index == MODEL_FRAMES: 
            index = 0;
        continue
    
    prev_frame = cur_frame
    cur_frame = gray
    
    if (avg is None): 
        if prev_frame is None: 
            continue
        
        print("Capturing Background Model");
        avg = prev_frame.copy().astype("float")
        continue
        
    # compute the absolute difference between the current frame and # first frame 
    frameDelta = cv2.absdiff(gray, cv2.convertScaleAbs(avg))
    DELTA_THRESH = 25;
    thresh = cv2.threshold(frameDelta, DELTA_THRESH, 255, cv2.THRESH_BINARY)[1]
    
    # dilate the thresholded image to fill in holes, then find contours # on thresholded image
    thresh = cv2.dilate(thresh, None, iterations=2) 
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) 
    cnts = imutils.grab_contours(cnts) # loop over the contours 
    
    prev_cnts = None;
    for c in cnts: # if the contour is too small, ignore it 
        if cv2.contourArea(c) < 100: #args["min_area"]: 
            continue 

        (x, y, w, h) = cv2.boundingRect(c) 
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2) 
        #print("Im before contor")
        if prev_cnts == None: #initialize the car id dictionary
            prev_cnts = {index:c for index, c in enumerate(cnts)}
            continue
            
        #print("Im in here now! Contour")
        similarity = {key : cv2.matchShapes(c, prev_cnts[key], 2, 0) for key in prev_cnts.keys()}
        temp = min(similarity.values()) 
        min_key = [key for key in similarity if similarity[key] == temp]
        cv2.putText(frame, "{}".format(min_key), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), thickness=2)

    # perform connected component labelling, using propagated
    cv2.imshow("Traffic Video", frame)  #show video
    if (cv2.waitKey(delay=1000) & 0xFF) == ord("q"):  #exit if user presses 'q'
        break 
    
    cv2.accumulateWeighted(gray, avg, 0.5) # accumulate average from current frame and previous

# exit video player and close all windows
vs.release() 
cv2.destroyAllWindows()
